In this Python project we will imporing table of list of postal codes of canada from wikipedia and clean the data

In [1]:
#importing necessary library
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#requesting the page of wikipedia
link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(link, 'html.parser')

In [3]:
#the required table
table=soup.find('table',class_='wikitable sortable')

In [4]:
#forming a meaningful dataframe from the lxml file
A = []
B = []
C = []

for row in table.find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).split('\n')[0])
        B.append(cells[1].find(text=True).split('\n')[0])
        C.append(cells[2].find(text=True).split('\n')[0])

In [5]:
#combining the various list to form the dataframe
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#grouping the dataframe based on the postal code and combining neighborhood having same postal code together
df["Neighborhood"] = df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

df = df.drop_duplicates()

if(df.index.name != 'Postal Code'):
    df =df.set_index('Postal Code')
    
df.head()

,Borough,Neighborhood
Postal Code,,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
#removing the not assigned borough
df = df[df['Borough'] != 'Not assigned']
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 2)

In [15]:
data = pd.read_csv(r'C:\Users\Hp\Downloads\Geospatial_Coordinates.csv')
data

,Postal Code,Latitude,Longitude
0,M3A,43.753259,-79.329656
1,M4A,43.725882,-79.315572
2,M5A,43.654260,-79.360636
3,M6A,43.718518,-79.464763
4,M7A,43.662301,-79.389494
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [19]:
complete_row=pd.merge(df, data, on='Postal Code')
complete_row

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
